# Process the data for the sepsis treatment experiment

Prerequisite:
 - Apply for the access of MIMIC III dataset here: https://mimic.mit.edu/iii/gettingstarted/
 - Follow the instruction in the AI Clinician repo to extract MIMICtable.mat (from file  AIClinician_mimic3_dataset_160219.m) https://gitlab.doc.ic.ac.uk/AIClinician/AIClinician
 - Run fixMIMICTable.m and extract_csv_data.m to preprocess the data to get MIMIC_outcome.csv table
 
What this file does:
 - generate the s45da_mimic_train_episodes, s45da_mimic_valid_episodes, s45da_mimic_test_episodes in the project_folder/data for the use of RL codes.

In [ ]:
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
import pickle
import matplotlib.pyplot as plt

In [ ]:
data_folder = "../../data"

In [ ]:
data = pd.read_csv('sepsis_matlab/MIMIC_outcome.csv')
for i in tqdm(range(data.shape[0])):
    if i != data.shape[0]-1 and (data['id'][i] == data['id'][i+1]):
        data.iloc[i,-1] = 0
    else:
        data.iloc[i,-1] = 100*(1-data.iloc[i,-1])

In [ ]:
patient_ids = data['id'].unique()
n_patients = patient_ids.shape[0]
print("Total number of patients : {}".format(n_patients))

In [ ]:
with open('sepsis_matlab/train_val_test_split.pkl', 'rb') as f:
    split_data = pickle.load(f)
    val_patient_id = split_data['val_patient_id']
    train_patient_id = split_data['train_patient_id']
    test_patient_id = split_data['test_patient_id']
n_train_patients = len(train_patient_id)
n_val_patients = len(val_patient_id)
n_test_patients = len(test_patient_id)
print(n_train_patients,n_val_patients,n_test_patients)

In [ ]:
class KNN_policy():
    def __init__(self, df, columns, k=100, weight=None):
        ''' Learning KNN policy with KNN 
        input
        -----
        df : dataframe
        k : int number of neighbours
        weight : weight vector 
        '''
        # Columns in df that are covaraites
        self.columns = columns
        if weight is None:
            weight = np.ones((len(self.columns), 1))
        self.weight = pd.DataFrame(weight).T
        self.weight.columns = self.columns
        self.weight = self.weight.iloc[0, :]
        
        self.X = df.loc[:, self.columns] # feature data frame
        self.A = df.loc[:, 'action'].to_numpy() # action data frame
        self.nA = 26 # number of actions : actions are indexed from 1 (Matalb :X)
        self.K = k
        self.whichA = np.ones((self.A.shape[0],self.nA))*np.inf
        self.whichA[np.arange(self.A.shape[0]),self.A] = 1
        
    def get_action_probability(self, x):
        if isinstance(x, pd.DataFrame):
            assert x.shape[0] == len(self.columns), 'Shape of iuput doesn\'t match, got {}'.format(x.shape)
        if isinstance(x, np.ndarray):
            assert x.shape[0] == len(self.columns), 'Shape of iuput doesn\'t match, got {}'.format(x.shape)
            x = pd.DataFrame(x).T
            x.columns = self.columns

        diff = (self.X - x.iloc[0, :])**2
        distance = (diff*self.weight).mean(axis=1)
        # indexes of 100 closes
        idxs = np.argsort(distance.to_numpy())[:self.K]
        actions = self.A[idxs]
        # action prob:
        action_prob = np.zeros(self.nA)
        for a in actions:
            action_prob[int(a)] += 1
        return action_prob/np.sum(action_prob)
    
    def get_nearest_same_action(self, x):
        if isinstance(x, pd.DataFrame):
            assert x.shape[0] == len(self.columns), 'Shape of iuput doesn\'t match, got {}'.format(x.shape)
        if isinstance(x, np.ndarray):
            assert x.shape[0] == len(self.columns), 'Shape of iuput doesn\'t match, got {}'.format(x.shape)
            x = pd.DataFrame(x).T
            x.columns = self.columns
        
        diff = (self.X - x.iloc[0, :])**2
        distance = (diff*self.weight).mean(axis=1)
        # indexes of 100 closes
        distance = distance.to_numpy()
        distance = distance*(self.whichA.transpose())
        action_dist = np.nanmin(distance,axis=-1)
        return action_dist

In [ ]:
COLUMNS = ['gender', 're_admission', 'mechvent', 'age', 'Weight_kg',
                   'GCS', 'HR', 'SysBP', 'MeanBP', 'DiaBP', 'RR', 'Temp_C', 'FiO2_1',
                   'Potassium', 'Sodium', 'Chloride', 'Glucose', 'Magnesium', 'Calcium',
                   'Hb', 'WBC_count', 'Platelets_count', 'PTT', 'PT', 'Arterial_pH',
                   'paO2', 'paCO2', 'Arterial_BE', 'Arterial_lactate', 'HCO3',
                   'Shock_Index', 'PaO2_FiO2', 'cumulated_balance', 'SOFA', 'SIRS', 'SpO2',
                   'BUN', 'Creatinine', 'SGOT', 'SGPT', 'Total_bili', 'INR',
                   'output_total', 'output_4hourly']

train_df = data.loc[data['id'].isin(train_patient_id)]
val_df = data.loc[data['id'].isin(val_patient_id)]
test_df = data.loc[data['id'].isin(test_patient_id)]

train_policy = KNN_policy(train_df, columns=COLUMNS, k=100)
val_policy = KNN_policy(val_df, columns=COLUMNS, k=100)
test_policy = KNN_policy(test_df, columns=COLUMNS, k=100)

In [ ]:
train_da = train_df.iloc[:,2:46].to_numpy()
val_da = val_df.iloc[:,2:46].to_numpy()
test_da = test_df.iloc[:,2:46].to_numpy()

## Get nearest neighbor distance with same action

In [ ]:
train_nearest_same_action = np.zeros((len(train_da), 25))

for i in tqdm(range(len(train_da))):
    train_nearest_same_action[i,:] = train_policy.get_nearest_same_action(train_da[i])[1:]

In [ ]:
np.save(f"{data_folder}/s45da_mimic_train_cur_nn_action_dist.npy", train_nearest_same_action)

In [ ]:
val_nearest_same_action = np.zeros((len(val_da), 25))

for i in tqdm(range(len(val_da))):
    val_nearest_same_action[i,:] = train_policy.get_nearest_same_action(val_da[i])[1:]

In [ ]:
np.save(f"{data_folder}/s45da_mimic_val_cur_nn_action_dist.npy", val_nearest_same_action)

In [ ]:
test_nearest_same_action = np.zeros((len(test_da), 25))

for i in tqdm(range(len(test_da))):
    test_nearest_same_action[i,:] = train_policy.get_nearest_same_action(test_da[i])[1:]

In [ ]:
np.save(f"{data_folder}/s45da_mimic_test_cur_nn_action_dist.npy", test_nearest_same_action)

## Get kNN probability

In [ ]:
# Computer (vectors of) probabilities for all states in dataset
def compute_pibs(df, knn):
    da = np.array(df)
    pibs = np.zeros((len(da), 25))
    
    for i in tqdm(range(len(da))):
        pibs[i,:] = knn.get_action_probability(da[i][2:46])[1:] 
        # kNN policy return values indexed from 1 !!!!!
    return pibs

In [ ]:
train_current_pibs = compute_pibs(train_df, train_policy)
np.save(f"{data_folder}/s45da_mimic_train_cur_pibs.npy", train_current_pibs)

In [ ]:
val_current_pibs = compute_pibs(val_df, val_policy)
val_estm_current_pibs = compute_pibs(val_df, train_policy)
np.save(f"{data_folder}/s45da_mimic_val_estm_cur_pibs.npy", val_estm_current_pibs)
np.save(f"{data_folder}/s45da_mimic_val_cur_pibs.npy", val_current_pibs)

In [ ]:
test_current_pibs = compute_pibs(test_df, test_policy)
test_estm_current_pibs = compute_pibs(test_df, train_policy)

In [ ]:
np.save(f"{data_folder}/s45da_mimic_test_cur_pibs.npy", test_current_pibs)
np.save(f"{data_folder}/s45da_mimic_test_estm_cur_pibs.npy", test_estm_current_pibs)

## Construct dataset

In [ ]:
def episodic_dataset(train_data, pibs, estm_pibs, nn_action_dist, obs_dim, n_patient, include_step=True):
    # Formatted the training set to be the replay buffer in BCQ/PQL code
    formatted_dataset = {'observations': np.zeros((n_patient,20,obs_dim-1)),
                         'actions': np.zeros((n_patient,20,1), np.int),
                         'rewards': np.zeros((n_patient,20,1)),
                         'not_done': np.zeros((n_patient,20,1)),
                         'pibs': np.zeros((n_patient,20,25)),
                         'estm_pibs': np.zeros((n_patient,20,25)),
                         'nn_action_dist': np.ones((n_patient,20,25))*1e9,
                        }
    
    n_eps = 0
    t = 0
    for i in range(0, len(train_data)):
        if include_step:
            formatted_dataset['observations'][n_eps,t,:] = np.array(train_data[i][1:obs_dim])
        else:
            formatted_dataset['observations'][n_eps,t,:] = np.array(train_data[i][2:obs_dim])
        formatted_dataset['actions'][n_eps,t,:] = train_data[i][obs_dim] - 1
        formatted_dataset['rewards'][n_eps,t,:] = train_data[i][obs_dim+1]
        formatted_dataset['pibs'][n_eps,t,:] = pibs[i,:]
        formatted_dataset['estm_pibs'][n_eps,t,:] = estm_pibs[i,:]
        formatted_dataset['nn_action_dist'][n_eps,t,:] = nn_action_dist[i,:]
        if i != len(train_data)-1 and train_data[i+1][0] == train_data[i][0]:
            # haven't moved to the next patient, same episode
            formatted_dataset['not_done'][n_eps,t,:] = 1
            t += 1
        else:
            formatted_dataset['not_done'][n_eps,t:,:] = 0
            formatted_dataset['pibs'][n_eps,t+1:,:] = 1.0
            formatted_dataset['estm_pibs'][n_eps,t+1:,:] = 1.0
            n_eps += 1
            t = 0
    print("Dataset with",n_eps,"episodes")
    return formatted_dataset

In [ ]:
obs_dimensions = 46

In [ ]:
train_dataset = episodic_dataset(train_df.to_numpy(), train_current_pibs, train_current_pibs, 
                                 train_nearest_same_action, obs_dimensions, len(train_patient_id))

In [ ]:
val_dataset = episodic_dataset(val_df.to_numpy(), val_current_pibs, val_estm_current_pibs, 
                               val_nearest_same_action, obs_dimensions, len(val_patient_id))

In [ ]:
test_dataset = episodic_dataset(test_df.to_numpy(), test_current_pibs, test_estm_current_pibs, 
                                test_nearest_same_action, obs_dimensions, len(test_patient_id))

In [ ]:
with open(f"{data_folder}/s45da_mimic_train_episodes", 'wb') as f:
    pickle.dump(train_dataset, f)

In [ ]:
with open(f"{data_folder}/s45da_mimic_val_episodes", 'wb') as f:
    pickle.dump(val_dataset, f)

In [ ]:
with open(f"{data_folder}/s45da_mimic_test_episodes", 'wb') as f:
    pickle.dump(test_dataset, f)